In [16]:
import torch 
import einops
import math
import re
import shutil

pth_file = "E:/project/neosr/experiments/otome_a4k_14_7_12/models/net_g_latest.pth"
hlsl_file = "C:/Users/khoi/Documents/Anime4K_Upscale_UUL.hlsl"
out_name = "otome_uul"

# hlsl_file = "C:/Users/khoi/Downloads/Magpie-dev-315f619/effects/Anime4K/Anime4K_Upscale_UL.hlsl"
# pth_file = "E:/project/neosr/old_experiments/test_a4k_ul_combine/models/net_g_latest.pth"
# out_name = "Anime4K_Upscale_UL_amakano"

# hlsl_file = "C:/Users/khoi/Downloads/Magpie-dev-315f619/effects/Anime4K/Anime4K_Upscale_UL.hlsl"
# pth_file = "E:/project/neosr/old_experiments/ul_hamidashi_huber_2/models/net_g_latest.pth"
# out_name = "Anime4K_Upscale_UL_hamidashi"


model = torch.load(pth_file, map_location=torch.device('cuda'))

def convert(weight, bias, data, doswap=False):
    swap = [0,2,1,3]
    out_chan, in_chan, width, height = weight.shape
    for to in range(math.ceil(out_chan/4)):
        for ti in range(math.ceil(in_chan/4)):
            for w in range(width):
                for h in range(height):
                    for i in range(min(4, in_chan)):
                        for o in range(min(4, out_chan)):
                            o = swap[o] if doswap else o
                            # data.append(float(weight[to*4+o, ti*4+i, w, h]))
                            data.append(float(weight[to*4+o, ti*4+i, h, w]))
        for o in range(min(4, out_chan)):
            o = swap[o] if doswap else o
            data.append(float(bias.data[to*4+o]))

if "params" in model.keys():
    model = model["params"]
layers = [i[:-7] for i in model.keys() if ".weight" in i]
data = []
for i in layers:
    # convert(model[i+".weight"], model[i+".bias"], data, doswap= "tail" in i)
    convert(model[i+".weight"], model[i+".bias"], data)


with open(hlsl_file,encoding='utf-8') as f:
    text = f.read()
data_iter = iter(data)
def replace_match(match):
    return str(next(data_iter))

pattern = r'-?\d+(\.\d{2,})(e-?\d+)?'

new_text = re.sub(pattern, replace_match, text)

with open(f"tmp/{out_name}.hlsl","w") as f:
    f.write(new_text)
    
try: 
	next(data_iter)
except StopIteration: 
	print("done")
else:
    print("error")
    
shutil.copy(f"tmp/{out_name}.hlsl","C:/Users/khoi/Downloads/Magpie-dev-f684374-x64/effects")

done


'C:/Users/khoi/Downloads/Magpie-dev-f684374-x64/effects\\otome_uul.hlsl'